# Banking and Unemployment
---
The below script explores the relationship between states with high unemployment rates and bank counts per state.

In this script, we retrieved and plotted data from the 2013 US Census and Google Places API to show the relationship between various socioeconomic parameters and bank count across 700 randomly selected zip codes. We used Pandas, Numpy, Matplotlib, Requests, Census API, and Google API to accomplish our task.

In [2]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states


# Census API Key
c = Census(census_key, year=2013)

## Data Retrieval

In [4]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E"), {
                         'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Employment Rate (Employment Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population,Poverty Rate
0,00683,33442.0,45.424915
1,00685,41947.0,55.009893
2,00687,32609.0,55.815879
3,00688,13340.0,54.017991
4,00690,6299.0,44.784886


## Combine Data

In [5]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_original = pd.read_csv(
    "../Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")

# Visualize
census_data_original.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count
0,624,"Hastings, MI 49058, USA",40.8,46777,22137,42.6306916,-85.2929384,9
1,692,"Ball, LA 71405, USA",35.8,55242,23941,31.4061799,-92.396174,12
2,730,"Great Mills, MD 20634, USA",31.9,79944,35961,38.2201614,-76.4967919,9
3,757,"Williamsport, TN 38487, USA",41.6,38125,18884,35.7310368,-87.2419299,0
4,957,"Marion, ND 58466, USA",44.5,69844,36981,46.5594224,-98.3481542,1


In [6]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(
    census_data_original, census_pd, how="left", on=["Zipcode", "Zipcode"])

# Remove rows missing data
census_data_complete = census_data_complete.dropna()

# Visualize
census_data_complete.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count,Population,Poverty Rate
63,10007,"Flatonia, TX 78941, USA",45.7,40304,23115,29.7574106,-97.1574784,0,6525.0,2.206897
64,10303,"Nevada, IA 50201, USA",40.4,56619,28908,42.065743,-93.4599326,7,24537.0,21.828259
65,10309,"Lukachukai, AZ 86507, USA",24.1,22009,8346,36.4106866,-109.2593642,0,32646.0,6.766526
66,10553,"Lone Pine, CA 93545, USA",40.6,32473,18444,36.5131184,-118.0888578,0,9895.0,15.260232
67,10803,"Niagara, WI 54151, USA",45.7,45813,23500,45.715354,-87.9804239,2,12439.0,3.006673


## Heatmap of poverty rate

In [7]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [8]:
# Store 'Lat' and 'Lng' into  locations 
locations = census_data_complete[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
poverty_rate = census_data_complete["Poverty Rate"].astype(float)

In [9]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Convert bank rate to list
bank_rate = census_data_complete["Bank Count"].tolist()

In [13]:
# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))